##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Classificação de imagem com o TensorFlow Lite Model Maker

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/image_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/image_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/image_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/lite/models/modify/model_maker/image_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
  <td>     <a href="https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo do TF Hub</a> </td>
</table>

A [biblioteca TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) (criador de modelos do TF Lite) simplifica o processo de adaptar e converter um modelo de rede neural do TensorFlow para dados de entrada específicos ao implantar esse modelo em aplicativos de aprendizado de máquina em dispositivos.

Este notebook apresenta um exemplo completo que utiliza a biblioteca Model Maker para ilustrar a adaptação e conversão de um modelo de classificação de imagens usado com frequência para classificar flores em um dispositivo móvel.

## Pré-requisitos

Para executar este exemplo, primeiro precisamos instalar diversos pacotes exigidos, incluindo o pacote do Model Maker que está no [repositório](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) do GitHub.

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker

Importe os pacotes necessários.

In [ ]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

## Exemplo completo simples

### Obtenha o caminho dos dados

Vamos obter alguns imagens para este exemplo completo simples. Algumas centenas de imagens são um bom ponto de partida para o Model Maker, enquanto mais dados poderão trazer uma exatidão maior.

In [ ]:
image_path = tf.keras.utils.get_file(
      'flower_photos.tgz',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      extract=True)
image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')

Você pode substituir `image_path` pelas duas pastas de imagens. Para carregar os dados no Colab, use o botão de upload disponível na barra lateral esquerda, conforme mostrado na imagem abaixo – o botão Upload está destacado com um retângulo vermelho. Tente carregar um arquivo ZIP e descompactá-lo. O caminho do arquivo raiz é o caminho atual.


<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_image_classification.png" width="800" hspace="100" alt="Upload File"> 

Se você preferir não carregar suas imagens na nuvem, pode tentar executar a biblioteca localmente de acordo com este [guia](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) no GitHub.

### Execute o exemplo

O exemplo tem apenas quatro linhas de código, conforme mostrado abaixo, em que cada uma representa uma etapa do processo geral.


Etapa 1 – Carregue os dados específicos para um aplicativo de aprendizado de máquina em dispositivos. Divida-os em dados de treinamento e de teste.

In [ ]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

Etapa 2 – Personalize o modelo do TensorFlow.

In [ ]:
model = image_classifier.create(train_data)

Etapa 3 – Avalie o modelo.

In [ ]:
loss, accuracy = model.evaluate(test_data)

Etapa 4 – Exporte para um modelo do TensorFlow Lite.

Aqui, exportamos para um modelo do TensorFlow Lite com [metadados](https://www.tensorflow.org/lite/models/convert/metadata), que fornecem um padrão para descrição de modelos. O arquivo de rótulos é incorporado aos metadados. Para a tarefa de classificação de imagens, a técnica padrão de quantização pós-treinamento é a quantização completa em inteiros.

Você pode baixá-lo pela barra lateral esquerda da mesma forma que fez para carregar os dados.

In [ ]:
model.export(export_dir='.')

Após essas quatro etapas simples, podemos usar o modelo do TensorFlow Lite em aplicativos em dispositivos, como no aplicativo de referência de [classificação de imagens](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification).

## Processo detalhado

No momento, temos suporte a diversos métodos, como EfficientNet-Lite*, MobileNetV2, ResNet50, como modelos pré-treinados para classificação de imagens, o que é muito flexível, pois é possível adicionar novos modelos pré-treinados a esta biblioteca com apenas algumas linhas de código.

Veja abaixo as etapas mais detalhadas deste exemplo.

### Etapa 1 – Carregue dados de entrada específicos em um aplicativo de aprendizado de máquina em dispositivos

O dataset Flower contém 3.670 imagens de 5 classes. Baixe a versão do arquivo do dataset e descompacte-a via tar.

O dataset tem a seguinte estrutura de diretórios:

<pre>
&lt;b&gt;flower_photos&lt;/b&gt;
|__ &lt;b&gt;daisy&lt;/b&gt;
    |______ 100080576_f52e8ee070_n.jpg
    |______ 14167534527_781ceb1b7a_n.jpg
    |______ ...
|__ &lt;b&gt;dandelion&lt;/b&gt;
    |______ 10043234166_e6dd915111_n.jpg
    |______ 1426682852_e62169221f_m.jpg
    |______ ...
|__ &lt;b&gt;roses&lt;/b&gt;
    |______ 102501987_3cdb8e5394_n.jpg
    |______ 14982802401_a3dfb22afb.jpg
    |______ ...
|__ &lt;b&gt;sunflowers&lt;/b&gt;
    |______ 12471791574_bb1be83df4.jpg
    |______ 15122112402_cafa41934f.jpg
    |______ ...
|__ &lt;b&gt;tulips&lt;/b&gt;
    |______ 13976522214_ccec508fe7.jpg
    |______ 14487943607_651e8062a1_m.jpg
    |______ ...
</pre>

In [ ]:
image_path = tf.keras.utils.get_file(
      'flower_photos.tgz',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      extract=True)
image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')

Use a classe `DataLoader` para carregar dados.

O método `from_folder()` pode carregar dados a partir da pasta. Ele pressupõe que os dados de imagem da mesma classe estejam no mesmo subdiretório e que o nome da subpasta seja o nome da classe. No momento, há suporte a imagens em JPEG e em PNG.

In [ ]:
data = DataLoader.from_folder(image_path)

Divida o dataset em dados de treinamento (80%), de validação (10%, opcional) e de teste (10%).

In [ ]:
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

Mostre 25 imagens de exemplo com seus respectivos rótulos.

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

### Etapa 2 – Personalize o modelo do TensorFlow

Crie um modelo de classificador de imagens personalizado com base nos dados carregados. O modelo padrão é EfficientNet-Lite0.


In [ ]:
model = image_classifier.create(train_data, validation_data=validation_data)

Confira a estrutura detalhada do modelo.

In [ ]:
model.summary()

### Etapa 3 – Avalie o modelo

Avalie o resultado do modelo; obtenha a perda e a exatidão do modelo.

In [ ]:
loss, accuracy = model.evaluate(test_data)

Podemos plotar os resultados previstos em 100 imagens de teste. Os rótulos previstos com a cor vermelha são os resultados previstos incorretamente, enquanto os outros são os previstos corretamente.

In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()

Se a exatidão não atender aos requisitos do aplicativo, consulte a seção [Uso avançado](#scrollTo=zNDBP2qA54aK) para ver alternativas, como alterar para um modelo maior, ajustar os parâmetros de retreinamento, etc.

### Etapa 4 – Exporte para um modelo do TensorFlow Lite

Converta o modelo treinado para o formato de modelos do TensorFlow Lite com [metadados](https://www.tensorflow.org/lite/models/convert/metadata) para poder usá-lo posteriormente em um aplicativo de aprendizado de máquina em dispositivos. O arquivo de rótulos e o arquivo de vocabulário são incorporados aos metadados. O nome de arquivo padrão do TF Lite é `model.tflite`.

Em diversos aplicativos de aprendizado de máquina em dispositivos, o tamanho do modelo é um fator importante. Portanto, recomendamos aplicar quantização no modelo para deixá-lo menor e possivelmente mais rápido. Para a tarefa de classificação de imagens, a técnica padrão de quantização pós-treinamento é a quantização completa em inteiros.

In [ ]:
model.export(export_dir='.')

Confira mais detalhes sobre como integrar o modelo do TensorFlow Lite aos aplicativos para dispositivos móveis nos [exemplos do guia](https://www.tensorflow.org/lite/examples/image_classification/overview) de classificação de imagens.

Este modelo pode ser integrado a um aplicativo para Android ou iOS usando a [API ImageClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/image_classifier) da [biblioteca Task do TensorFlow Lite](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview).

Confira abaixo os formatos de exportação permitidos:

- `ExportFormat.TFLITE`
- `ExportFormat.LABEL`
- `ExportFormat.SAVED_MODEL`

Por padrão, só é exportado o modelo do TensorFlow Lite com metadados. Você também pode exportar diferentes arquivos seletivamente. Por exemplo: exporte somente o arquivo de rótulos da seguinte forma:

In [ ]:
model.export(export_dir='.', export_format=ExportFormat.LABEL)

E você pode avaliar o modelo do TF Lite com o método `evaluate_tflite`.

In [ ]:
model.evaluate_tflite('model.tflite', test_data)

## Uso avançado

A função `create` é uma parte essencial dessa biblioteca e usa aprendizado por transferência com um modelo pré-treinado similar ao do [tutorial](https://www.tensorflow.org/tutorials/images/transfer_learning).

A função `create` consiste nas seguintes etapas:

1. Divide os dados em dados de treinamento, validação e teste de acordo com os parâmetros `validation_ratio` e `test_ratio`. O valor padrão de `validation_ratio` e `test_ratio` é `0.1` e `0.1`.
2. Baixa um [vetor de características de imagens](https://www.tensorflow.org/hub/common_signatures/images#image_feature_vector) como modelo base no TensorFlow Hub. O modelo pré-treinado padrão é EfficientNet-Lite0.
3. Adiciona um head de classificador com uma camada de dropout, com `dropout_rate` entre a camada do head e o modelo pré-treinado. O `dropout_rate` padrão é o valor `dropout_rate` padrão de [make_image_classifier_lib](https://github.com/tensorflow/hub/blob/master/tensorflow_hub/tools/make_image_classifier/make_image_classifier_lib.py#L55) do TensorFlow Hub.
4. Faz o pré-processamento dos dados de entrada não tratados. No momento, as etapas de pré-processamento incluem a normalização do valor de cada pixel da imagem para a escala da entrada do modelo e o redimensionamento para o tamanho da entrada do modelo. EfficientNet-Lite0 tem escala de entrada igual a `[0, 1]` e tamanho da imagem de entrada igual a `[224, 224, 3]`.
5. Alimenta o modelo do classificador com dados. Por padrão, os parâmetros de treinamento, como épocas de treinamento, tamanho do lote, taxa de aprendizado e momento, são os valores padrão de [make_image_classifier_lib](https://github.com/tensorflow/hub/blob/master/tensorflow_hub/tools/make_image_classifier/make_image_classifier_lib.py#L55) do TensorFlow Hub. Somente o head do classificador é treinado.

Nesta seção, descreveremos diversos tópicos avançados, incluindo como alterar para um modelo de classificação de imagens diferente, como mudar os hiperparâmetros de treinamento, etc.


## Personalize a quantização pós-treinamento em um modelo do TensorFlow Lite


A [quantização pós-treinamento](https://www.tensorflow.org/lite/performance/post_training_quantization) é uma técnica de conversão que pode reduzir o tamanho do modelo e a latência de inferência, além de aumentar a velocidade de inferência da CPU e do acelerador de hardware com uma pequena redução da exatidão do modelo. A quantização é amplamente utilizada para otimizar o modelo.


A biblioteca Model Maker aplica uma técnica padrão de quantização pós-treinamento ao exportar o modelo. Se você quiser personalizar a quantização pós-treinamento, o Model Maker oferece suporte a diversas opções usando [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig). Vejamos a quantização de float16 como exemplo. Primeiro, definimos a configuração de quantização.

In [ ]:
config = QuantizationConfig.for_float16()

Em seguida, exportamos o modelo do TensorFlow Lite com essa configuração.

In [ ]:
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)

No Colab, você pode baixar o modelo com nome `model_fp16.tflite` pela barra lateral esquerda da mesma forma que fez para carregar dados anteriormente.

## Altere o modelo


### Altere para o modelo compatível com essa biblioteca.

No momento, a biblioteca oferece suporte a modelos EfficientNet-Lite, MobileNetV2, ResNet50. [EfficientNet-Lite](https://github.com/tensorflow/tpu/tree/master/models/official/efficientnet/lite) é uma família de modelos de classificação de imagens que atingem uma exatidão excepcional e são adequadas para dispositivos Edge. O modelo padrão é EfficientNet-Lite0.

Para mudar o modelo para MobileNetV2, basta definir o parâmetro `model_spec` como a especificação do modelo MobileNetV2 no método `create`.

In [ ]:
model = image_classifier.create(train_data, model_spec=model_spec.get('mobilenet_v2'), validation_data=validation_data)

Avalie o modelo MobileNetV2 retreinado recentemente para ver a exatidão e a perda com os dados de teste.

In [ ]:
loss, accuracy = model.evaluate(test_data)

### Altere para o modelo do TensorFlow Hub

Além disso, também podemos alterar para outros modelos novos que recebem uma imagem como entrada e geram como saída um vetor de características no formato do TensorFlow Hub.

Usando o modelo [Inception V3](https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1) como exemplo, podemos definir `inception_v3_spec`, que é um objeto de [image_classifier.ModelSpec](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/image_classifier/ModelSpec) e contém a especificação do modelo Inception V3.

Precisamos especificar o nome `name` e a URL `uri` do modelo do TensorFlow Hub. O valor padrão de `input_image_shape` é `[224, 224]`. Precisamos alterá-lo para `[299, 299]` para o modelo Inception V3.

In [ ]:
inception_v3_spec = image_classifier.ModelSpec(
    uri='https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1')
inception_v3_spec.input_image_shape = [299, 299]

Em seguida, ao definir o parâmetro `model_spec` como `inception_v3_spec` no método `create`, podemos retreinar o modelo Inception V3.

As etapas restantes são exatamente iguais, e poderemos ter um modelo InceptionV3 personalizado do TensorFlow Lite no final.

### Altere seu próprio modelo personalizado

Se quisermos usar um modelo personalizado que não esteja no TensorFlow Hub, precisamos criar e exportar [ModelSpec](https://www.tensorflow.org/hub/api_docs/python/hub/ModuleSpec) no TensorFlow Hub.

Começamos definindo o objeto `ModelSpec` conforme o processo acima.

## Altere os hiperparâmetros de treinamento

Também podemos alterar os hiperparâmetros de treinamento, como `epochs`, `dropout_rate` e `batch_size`, o que pode afetar a exatidão do modelo. Veja quais parâmetros do modelo podem ser ajustados:

- `epochs`: mais épocas podem alcançar uma exatidão melhor até a convergência, mas treinar com épocas demais pode causar overfitting.
- `dropout_rate`: taxa de dropout, evita overfitting. É igual a none (nenhuma) por padrão.
- `batch_size`: número de amostras a serem usadas em um passo de treinamento. É igual a none (nenhum) por padrão.
- `validation_data`: dados de validação. Se igual a none (nenhum), pula o processo de validação. É igual a none por padrão.
- `train_whole_model`: se verdadeiro, o módulo do Hub é treinado junto com a camada de classificação superior. Caso contrário, treina somente a camada de classificação superior. É igual a none (nenhum) por padrão.
- `learning_rate`: Taxa de aprendizado base. É igual a none (nenhuma) por padrão.
- `momentum`: float do Python encaminhado ao otimizador. É usado somente quando `use_hub_library` é true (verdadeiro). É igual a none (nenhum) por padrão.
- `shuffle`: booleano, indica se os dados devem ser misturados. É igual a false (falso) por padrão.
- `use_augmentation`: booleano, indica o uso de ampliação de dados no pré-processamento. É igual a false (falso) por padrão.
- `use_hub_library`: booleano, usa `make_image_classifier_lib` do TensorFlow Hub para retreinar o modelo. Este pipeline de treinamento pode alcançar um desempenho melhor para datasets complicados com muitas categorias. É igual a true (verdadeiro) por padrão.
- `warmup_steps`: número de etapas de preparação para o cronograma de preparação da taxa de aprendizado. Se igual a none (nenhum), o warmup_steps padrão é usado, que é o total de passos de treinamento em duas épocas. É usado somente quando `use_hub_library` é false (falso). É igual a none (nenhum) por padrão.
- `model_dir`: opcional, é o local dos arquivos de checkpoint do modelo. É usado somente quando `use_hub_library` é false (falso). É igual a none (nenhum) por padrão.

Os parâmetros que são iguais a none (nenhum) por padrão, como `epochs`, obterão os parâmetros concretos padrão em [make_image_classifier_lib](https://github.com/tensorflow/hub/blob/02ab9b7d3455e99e97abecf43c5d598a5528e20c/tensorflow_hub/tools/make_image_classifier/make_image_classifier_lib.py#L54) da biblioteca do TensorFlow Hub ou em [train_image_classifier_lib](https://github.com/tensorflow/examples/blob/f0260433d133fd3cea4a920d1e53ecda07163aee/tensorflow_examples/lite/model_maker/core/task/train_image_classifier_lib.py#L61).

Por exemplo: podemos treinar com mais épocas.


In [ ]:
model = image_classifier.create(train_data, validation_data=validation_data, epochs=10)

Avalie o modelo retreinado recentemente com 10 épocas.

In [ ]:
loss, accuracy = model.evaluate(test_data)

# Saiba mais

Leia o exemplo de [classificação de imagens](https://www.tensorflow.org/lite/examples/image_classification/overview) para aprender os detalhes técnicos. Confira mais informações em:

- [Guia](https://www.tensorflow.org/lite/models/modify/model_maker) e [referência da API](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker) do TensorFlow Lite Model Maker.
- Task Library: [ImageClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/image_classifier) para implantação.
- Aplicativos de referência completos para [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/android), [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/ios) e [Raspberry PI](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/raspberry_pi).
